# Demo: Spatial reasoning using WiFi Fingerprinting

Course: NICF- Spatial Reasoning from Sensor Data (SF)

Website: https://www.iss.nus.edu.sg/executive-education/course/detail/spatial-reasoning-from-sensor-data/artificial-intelligence

Contact: Dr. Tian Jing

Email: tianjing@nus.edu.sg


## Objective

- Perform spatial reasoning using WiFi fingerprinting

## Installation guideline

- Open `Anaconda Prompt`

- Append the channel `conda-forge` into your conda configuration.

`conda config --append channels conda-forge`

- Create a new virtual environment `srsdv` or install additional packages in your own environment

**[Windows, CPU version]**

`conda create -n srsdv python=3.6 numpy=1.15.1 opencv=3.4.2 matplotlib=2.2.3 tensorflow=1.12.0 scipy=1.1.0 scikit-learn=0.19.1 spyder=3.3.2 yaml=0.1.7 keras=2.2.4 pillow=5.4.1 notebook=5.7.4 pandas=0.24.2 h5py=2.8.0`

**[Windows, GPU version, CUDA 9.0]**

`conda create -n srsdv python=3.6 numpy=1.15.1 opencv=3.4.2 matplotlib=2.2.3 tensorflow-gpu=1.12.0 scipy=1.1.0 scikit-learn=0.19.1 spyder=3.3.2 yaml=0.1.7 keras-gpu=2.2.4 pillow=5.4.1 notebook=5.7.4 pandas=0.24.2 h5py=2.8.0`

- Activate the environment `srsdv`

`conda activate srsdv`

- Browse to the folder that contains the workshop files, then run `Jupyter Notebook`

`jupyter notebook`


## Reference

- Dataset: UJIIndoorLoc Dataset: https://www.kaggle.com/giantuji/UjiIndoorLoc


In [1]:

import numpy as np
import os
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
import tensorflow as tf

print("Tensorflow version: ", tf.__version__)
print(tf.test.gpu_device_name())


Tensorflow version:  1.12.0
/device:GPU:0


In [2]:
path_train = "UJIIndoorLoc_trainingData.csv"
path_validation = "UJIIndoorLoc_validationData.csv"

#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.iloc[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths).astype('float64'))

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)
class_label = np.unique(train_labels)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)

# one hot encode the dummy_labels. this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) as the column names

train_labels = np.asarray(dummy_labels)

print(train_labels.shape)

(19930, 13)


In [3]:
# Split dataset
train_val_split = np.random.rand(len(train_AP_features))
train_val_split = train_val_split < 0.70 #should contain ~70% percent true
# We will then split our given training set into training + validation 
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]


In [4]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.iloc[:,0:520]))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

C:\Users\isstj\AppData\Local\Continuum\anaconda3\envs\kanchi\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [5]:
nb_epochs = 10
batch_size = 10
input_size = 520
num_classes = 13

In [6]:
#Define model
def encoder():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', use_bias=True))
    model.add(Dense(128, activation='tanh', use_bias=True))
    model.add(Dense(64, activation='tanh', use_bias=True))
    return model

def decoder(e):   
    e.add(Dense(128, input_dim=64, activation='tanh', use_bias=True))
    e.add(Dense(256, activation='tanh', use_bias=True))
    e.add(Dense(input_size, activation='tanh', use_bias=True))
    e.compile(optimizer='adam', loss='mse')
    return e
    
e = encoder()
e.summary()

d_pre = decoder(e)
d_pre.summary()
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               133376    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
Total params: 174,528
Trainable params: 174,528
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               133376    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dens

In [7]:
hist = d_pre.fit(train_X, train_X, epochs=nb_epochs, batch_size=batch_size)

Epoch 1/10
14057/14057 [==============================] - 5s 390us/step - loss: 0.7065
Epoch 2/10
14057/14057 [==============================] - 5s 326us/step - loss: 0.6827
Epoch 3/10
14057/14057 [==============================] - 5s 325us/step - loss: 0.6772
Epoch 4/10
14057/14057 [==============================] - 5s 325us/step - loss: 0.6743
Epoch 5/10
14057/14057 [==============================] - 5s 327us/step - loss: 0.6721
Epoch 6/10
14057/14057 [==============================] - 5s 329us/step - loss: 0.6702
Epoch 7/10
14057/14057 [==============================] - 5s 326us/step - loss: 0.6687
Epoch 8/10
14057/14057 [==============================] - 5s 326us/step - loss: 0.6671
Epoch 9/10
14057/14057 [==============================] - 5s 326us/step - loss: 0.6657
Epoch 10/10
14057/14057 [==============================] - 5s 325us/step - loss: 0.6647


In [8]:
# Based on trained autoencoder features, define a classification model

def classifier(d):
    num_to_remove = 3
    for i in range(num_to_remove):
        d.pop()
    d.add(Dense(128, input_dim=64, activation='tanh', use_bias=True))
    d.add(Dense(128, activation='tanh', use_bias=True))
    d.add(Dense(num_classes, activation='softmax', use_bias=True))
    d.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return d

c = classifier(d_pre)
c.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=nb_epochs, batch_size=batch_size)
c.summary()


Train on 14057 samples, validate on 5873 samples
Epoch 1/10
14057/14057 [==============================] - 6s 437us/step - loss: 0.3199 - acc: 0.8872 - val_loss: 0.2036 - val_acc: 0.9271
Epoch 2/10
14057/14057 [==============================] - 6s 407us/step - loss: 0.1471 - acc: 0.9505 - val_loss: 0.1476 - val_acc: 0.9470
Epoch 3/10
14057/14057 [==============================] - 6s 404us/step - loss: 0.1104 - acc: 0.9628 - val_loss: 0.2029 - val_acc: 0.9321
Epoch 4/10
14057/14057 [==============================] - 6s 405us/step - loss: 0.0948 - acc: 0.9676 - val_loss: 0.1645 - val_acc: 0.9508
Epoch 5/10
14057/14057 [==============================] - 6s 406us/step - loss: 0.0788 - acc: 0.9726 - val_loss: 0.1834 - val_acc: 0.9411
Epoch 6/10
14057/14057 [==============================] - 6s 404us/step - loss: 0.0797 - acc: 0.9739 - val_loss: 0.1614 - val_acc: 0.9516
Epoch 7/10
14057/14057 [==============================] - 6s 408us/step - loss: 0.0698 - acc: 0.9777 - val_loss: 0.1734 - v

In [9]:
# Evaluate the performance

loss, acc = c.evaluate(test_AP_features, test_labels, verbose=0)
print('Accuracy: %.4f\n' % acc)

# Get the class id from the one-hot encoding
predict_labels = c.predict(test_AP_features)
predict_class = np.argmax(predict_labels, axis=1)
test_class = np.argmax(test_labels, axis=1)
# Display the confusion matrix
print(pd.DataFrame(confusion_matrix(predict_class, test_class), index=class_label, columns=class_label))

Accuracy: 0.7381

    00   01   02  03  10   11  12  13  20   21  22  23  24
00  67   59    3   1   0    0   0   0   0    0   0   0   0
01   6  127    8   0   0    0   0   0   0    0   0   0   0
02   3   12  112  10   0    0   0   0   0    0   0   0   0
03   0    0   41  71   0    0   0   0   0    1   0   0   0
10   0    1    0   0  21   16   1   0   0    0   0   0   1
11   0    0    0   0   7  101   2   0   0    0   0   0   0
12   0    0    0   1   1   24  61  12   0    0   0   0   0
13   2    4    0   0   0    2  22  34   0    0   0   0   0
20   0    0    0   0   0    0   1   0  22    4   0   0   0
21   0    1    1   2   0    0   0   0   2  100   3   0   0
22   0    3    0   0   0    0   0   0   0    5  41   0   0
23   0    1    0   0   1    0   0   0   0    1  10  36  11
24   0    0    0   0   0    0   0   1   0    0   0   4  27


Have a nice day!